# SetUp

In [ ]:
from pandas import read_csv
from datetime import datetime
import numpy as np
import pandas as pd
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from google.colab import files

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg
 


# Upload Data

In [ ]:
# load data
from google.colab import files
uploaded = files.upload()

In [ ]:


# load dataset
dataset = read_csv('DataSetWater.csv', header=0, index_col=0)
dataset = dataset.drop(columns=['Q25(t)','Q25(t-1)','Q25(t-2)','Q25(t-3)','Q25(t-4)','Rain25'])
values = dataset.values
# specify columns to plot
groups = [0, 1, 2, 3,4,5]
i = 1
# plot each column
pyplot.figure()
for group in groups:
	pyplot.subplot(len(groups), 1, i)
	pyplot.plot(values[:, group])
	pyplot.title(dataset.columns[group], y=0.5, loc='right')
	i += 1
pyplot.show()

In [ ]:
# integer encode direction
encoder = LabelEncoder()
values[:,5] = encoder.fit_transform(values[:,5])
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed4h = series_to_supervised(scaled, 1, 4)

# drop columns we don't want to predict 

reframed4h.drop(reframed4.columns[[6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,25,26,27,28,29]], axis=1, inplace=True)

# Create Data to Train and Test

In [ ]:
reframed=reframed4h

In [ ]:
def fit_lstm(train, batch_size, nb_epoch, neurons):
	X, y = train[:, 0:-1], train[:, -1]
	X = X.reshape(X.shape[0], 1, X.shape[1])
	model = Sequential()
	model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
	model.add(Dense(1))
	model.compile(loss='mean_squared_error', optimizer='adam')

	for i in range(nb_epoch):
		model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
		model.reset_states()
	return model

In [ ]:
values = reframed.values
n_train_hours = int(len(reframed)*0.8)
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
  # reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

In [ ]:
repeats = 1
error_MAPE_train = list()
error_MAPE_test = list()
error_rmse_train = list()
error_rmse_test = list()
model_list = list()
output_train=pd.DataFrame(train_y)
output_test=pd.DataFrame(test_y)

In [ ]:

for r in range(repeats):
  # lstm_model = fit_lstm(train, 1, 100, 6)
  # -----------------------------------------------------------------------
  values = reframed.values
  # -----------------------------------------------------------------------
  n_train_hours = int(len(reframed)*0.8)
  train = values[:n_train_hours, :]
  test = values[n_train_hours:, :]
  # split into input and outputs
  train_X, train_y = train[:, :-1], train[:, -1]
  test_X, test_y = test[:, :-1], test[:, -1]
  # reshape input to be 3D [samples, timesteps, features]
  train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
  test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
  print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

  model = Sequential()
  model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
  model.add(Dense(units=1, activation='linear'))
  model.compile(optimizer='adam', loss='mean_squared_error')
  # fit network
  history = model.fit(train_X, train_y, epochs=50, batch_size=128, validation_data=(test_X, test_y), verbose=2, shuffle=False)

  model_list.append(model)
  

  # make Train
  yhat = model.predict(train_X)
  train_X = train_X.reshape((train_X.shape[0], train_X.shape[2]))
  # invert scaling for forecast
  inv_yhat_train = concatenate((yhat, train_X), axis=1)
  inv_yhat_train = scaler.inverse_transform(inv_yhat_train)
  inv_yhat_train = inv_yhat_train[:,0]
  
  # invert scaling for actual
  train_y = train_y.reshape((len(train_y), 1))
  inv_y_train = concatenate((train_y, train_X[:, 1:]), axis=1)
  inv_y_train = scaler.inverse_transform(inv_y_train)
  inv_y_train = inv_y_train[:,0]
  # calculate RMSE
  # print(inv_y_train)
  output_train[r+1]=pd.DataFrame(inv_yhat_train)
  
  rmse_train = sqrt(mean_squared_error(inv_y_train, inv_yhat_train))
  MAPE_train=mean_absolute_percentage_error(inv_y_train, inv_yhat_train)*100
  # -----------------------------------------------------------------------
  print("===========================================>", r )
  # -----------------------------------------------------------------------

  # make a prediction
  yhat = model.predict(test_X)
  test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
  # invert scaling for forecast
  inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
  inv_yhat = scaler.inverse_transform(inv_yhat)
  inv_yhat = inv_yhat[:,0]
  # invert scaling for actual
  test_y = test_y.reshape((len(test_y), 1))
  inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
  inv_y = scaler.inverse_transform(inv_y)
  inv_y = inv_y[:,0]
  output_test[r+1]=pd.DataFrame(inv_yhat)
  # calculate RMSE
  rmse_test = sqrt(mean_squared_error(inv_y, inv_yhat))
  MAPE_test=mean_absolute_percentage_error(inv_y, inv_yhat)*100

  error_MAPE_train.append(MAPE_train)
  error_MAPE_test.append(MAPE_test)
  error_rmse_train.append(rmse_train)
  error_rmse_test.append(rmse_test)


output_train["real"]=pd.DataFrame(inv_y_train)
output_test["real"]=pd.DataFrame(inv_y)
                                                                                     

In [ ]:
 error_MAPE_train

# Make File to keep the output

In [ ]:
output_error = pd.DataFrame(error_MAPE_train)
output_error.to_csv('output26_error_500_t4.csv')

In [ ]:
# summarize results
results = DataFrame()
results['MAPE'] = error_MAPE_train
print(results.describe())
results.boxplot()
pyplot.show()

In [ ]:
output_train.to_csv('output_train_sti26_500_t4.csv')
output_test.to_csv('output_test_sti26_500_t4.csv')

In [ ]:
from keras.models import load_model
model_list[3].save('model_best26_t4.h5')

In [ ]:
# model = load_model('model_best25.h5')